In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
csv = pd.read_csv("healthcare-dataset-stroke-data\healthcare-dataset-stroke-data.csv",na_values="Unknown",encoding='utf-8')
csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     3566 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [2]:
x_columns = csv.loc[csv['bmi'].notna(), (csv.columns != 'bmi') & (csv.columns != 'id')]
x_columns.head(5)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,formerly smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,formerly smoked,1


In [3]:
y_column = csv.loc[csv['bmi'].notna(),'bmi']
y_column.head(5)

0    36.6
2    32.5
3    34.4
4    24.0
5    29.0
Name: bmi, dtype: float64

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
str_columns = x_columns.select_dtypes(include=object)
x_columns.loc[:,'gender'] = le.fit_transform(x_columns["gender"])
x_columns.loc[:,'ever_married'] = le.fit_transform(x_columns["ever_married"])
x_columns.loc[:,'work_type'] = le.fit_transform(x_columns["work_type"])
x_columns.loc[:,'Residence_type'] = le.fit_transform(x_columns["Residence_type"])
x_columns.loc[:,'smoking_status'] = le.fit_transform(x_columns["smoking_status"])
x_columns.loc[:,'age'] = le.fit_transform(x_columns["age"])
x_columns.loc[:,'avg_glucose_level'] = le.fit_transform(x_columns["avg_glucose_level"])
y_column = le.fit_transform(y_column)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_columns, y_column, random_state=42)

In [6]:
## Import the Classifier.
from sklearn.neighbors import KNeighborsClassifier
## Instantiate the model with 5 neighbors. 
knn = KNeighborsClassifier(n_neighbors=5)
## Fit the model on the training data.
knn.fit(X_train, y_train)
## See how the model performs on the test data.
knn.score(X_test, y_test)


0.004885993485342019

In [7]:
csv.loc[:,'gender'] = le.fit_transform(csv["gender"])
csv.loc[:,'ever_married'] = le.fit_transform(csv["ever_married"])
csv.loc[:,'work_type'] = le.fit_transform(csv["work_type"])
csv.loc[:,'Residence_type'] = le.fit_transform(csv["Residence_type"])
csv.loc[:,'smoking_status'] = le.fit_transform(csv["smoking_status"])
csv.loc[:,'age'] = le.fit_transform(csv["age"])
csv.loc[:,'avg_glucose_level'] = le.fit_transform(csv["avg_glucose_level"])

tempCsv = csv.copy(deep=True)
tempCsv.drop(labels=['bmi','id'],axis=1,inplace=True)
newBmi = knn.predict(tempCsv)

In [8]:
import numpy as np
csv['bmi'].fillna(value=pd.Series(newBmi),inplace=True) 
csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   int32  
 2   age                5110 non-null   int64  
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   int32  
 6   work_type          5110 non-null   int32  
 7   Residence_type     5110 non-null   int32  
 8   avg_glucose_level  5110 non-null   int64  
 9   bmi                5110 non-null   float64
 10  smoking_status     5110 non-null   int32  
 11  stroke             5110 non-null   int64  
dtypes: float64(1), int32(5), int64(6)
memory usage: 379.4 KB


In [9]:
# Use numpy to convert to arrays
import numpy as np

# Labels are the values we want to predict
labels = np.array(csv['stroke'])
# Remove the labels from the csv
# axis 1 refers to the columns
csv= csv.drop('stroke', axis = 1)
csv= csv.drop('id', axis = 1)
# Saving feature names for later use
feature_list = list(csv.columns)
# Convert to numpy array
csv = np.array(csv)

In [10]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(csv, labels, test_size = 0.25, random_state = 42)


In [11]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [12]:
predictions = rf.predict(test_features)
predictions = predictions.round()


In [13]:
from sklearn.metrics import f1_score
print(f1_score(test_labels, predictions, average=None))
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(test_labels, predictions)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

[0.96881328 0.11494253]
Average precision-recall score: 0.10
